In [1]:
import tensorflow as tf
import pandas as pd
import string
import numpy as np
import nltk
from nltk import word_tokenize
from tensorflow import keras

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


# Load the Dataset

In [2]:
data = pd.read_csv('data.csv',encoding='utf8')
# del data['Unnamed: 0'] # Removing the index column
data

,Unnamed: 0,text,label
0,0,"GAZA/CAIRO (Reuters) - Palestinian factions, i...",1
1,1,HARARE (Reuters) - Zimbabwean police arrested ...,1
2,2,Ronna Romney McDaniel is the Chairman of the M...,0
3,3,WASHINGTON (Reuters) - A small group of Republ...,1
4,4,"THE HUNTINGTON BEACH, CA RALLY WAS PRETTY BIG ...",0
...,...,...,...
44893,44893,There are two small problems with your analogy...,0
44894,44894,Consortium News Exclusive: The U.S. mainstream...,0
44895,44895,Just as drug addicts and alcoholics tend to be...,0
44896,44896,WASHINGTON (Reuters) - Republican presidential...,1


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  44898 non-null  int64 
 1   text        44898 non-null  object
 2   label       44898 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.0+ MB


# Data Preprocessing

### Spliting the data

In [4]:
# Training and Validation Split
validation_split = 0.33
num_validation_samples = int(validation_split * len(data))
train_samples = data["text"][:-num_validation_samples]
val_samples = data["text"][-num_validation_samples:]
train_labels = data["label"][:-num_validation_samples]
val_labels = data["label"][-num_validation_samples:]

### Word Vector Representation

In [5]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

Length_of_input_sentenses=500

# matrix of pretrained weights
vectorizer = TextVectorization(max_tokens=50000, output_sequence_length=500)
text_ds = tf.data.Dataset.from_tensor_slices(train_samples)
vectorizer.adapt(text_ds)


In [6]:
# Customize get_vocabulary method
def _get_vocabulary():
    keys, values = vectorizer._index_lookup_layer._table_handler.data()
    return [x.decode('utf8', errors='ignore') for _, x in sorted(zip(values, keys))]

In [7]:
# vectorizer.get_vocabulary()
_get_vocabulary()[:5]

['', 'the', 'to', 'of', 'a']

In [8]:
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([    2,  8804,  3275,     9,     2, 17306], dtype=int64)

In [9]:
voc = _get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

embeddings_index = {}
with open('glove.6B.100d.txt', encoding="utf8") as f:
    for line in f:
        
        word, coefs = line.split(maxsplit=1)
        
        coefs = np.fromstring(coefs, "f", sep=" ")
        
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [10]:
num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 38174 words (11811 misses)


# Embedding Layer

In [11]:
from tensorflow.keras.layers import Embedding

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

# Model

In [12]:
from tensorflow.keras import layers

In [19]:
model = keras.Sequential()
model.add(embedding_layer)
model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(32, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [14]:
# Training and Testing split
# validation_split = 0.2
# num_validation_samples = int(validation_split * len(train_samples))
# x_val = np.array(train_samples[:-num_validation_samples])
# x_train = np.array(train_samples[-num_validation_samples:])
# y_val = np.array(train_labels[:-num_validation_samples])
# y_train = np.array(train_labels[-num_validation_samples:])

x_train = vectorizer(np.array([[s] for s in train_samples])).numpy()
x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
y_val = np.array(val_labels)

In [18]:
model.compile(loss="mse", optimizer="adam", metrics=["acc"])
model.fit(x_train, y_train, batch_size=128, epochs=50, validation_data=(x_val, y_val))

Epoch 1/50
104/236 [============>.................] - ETA: 1:12 - loss: 0.2308 - acc: 0.5941

KeyboardInterrupt: 